In [40]:
import random
from TwoWayDict import *
from epidemicsim_functions import *
id_coordindex = TwoWayDict()
grid_list = []
active, recovered, total_infected = 1, 0, 1

In [41]:
def scatter_uniform(N,M,density=0.36):
    global grid_list
    global active
    global recovered
    global total_infected
    active, recovered, total_infected = 1, 0, 1
    grid_list.clear()
    id_coordindex.__clear__()
    grid = create_grid(N, M)
    for i in range(len(grid)):
        grid_list.append([])
    for i in range(len(grid)):
        grid_list[i] = list(grid[i]) # ndarray 형식을 list 형식으로 변환
    for i in range(1, len(grid) - 1):
        for j in range(1, len(grid[0]) - 1):
            grid_list[i][j] = [] # 빈 리스트로 초기화(사람이 없는 칸)
    num_entity = int(N * M * density) # 개체의 수
    k = 0 # id 초기화
    for n in random.sample(range(1, N * M + 1), num_entity):
        i, j = n // M, n % M # n번째 점의 2차원 좌표
        if j == 0:
            grid_list[i][M].append([k, 0])
            id_coordindex[k] = ((i, M), 0)
        else:
            grid_list[i + 1][j].append([k, 0])
            id_coordindex[k] = ((i + 1, j), 0)
        k += 1
    # 초기 감염자 위치 설정: 중앙
    mid_x, mid_y = (N + 1) // 2, (M + 1) // 2
    if ((mid_x, mid_y), 0) not in id_coordindex:
        grid_list[mid_x][mid_y].append([k, 0])
        id_coordindex[k] = ((mid_x, mid_y), 0)
    return grid_list

In [42]:
# # Initialize
# grid_list = scatter_uniform(5,6)

In [43]:
# print(grid_list)
# print(id_coordindex)
# print(len(id_coordindex))
# for i in range(len(id_coordindex)):
#     print(id_coordindex[i][0][0], id_coordindex[i][0][1], id_coordindex[i][1])

In [44]:
def move_uniform():
    global grid_list
    for i in range(id_coordindex.__len__()) : # 식별번호(id)들에 대해서
#         print(i)
        x_before, y_before = id_coordindex[i][0][0], id_coordindex[i][0][1] # 처음의 x, y좌표
        index_before = id_coordindex[i][1] # 처음의 위치에서의 index
#         print(x_before, y_before, index_before)
#         print(grid_list)
        id_status = grid_list[x_before][y_before][index_before] # 개체의 식별번호(id)와 상태(status)
        x_movement, y_movement = random.randint(-1, 1), random.randint(-1, 1) # 최대 1칸 이동
        x_after, y_after = x_before + x_movement, y_before + y_movement # 나중의 x, y좌표
        how_many_in_before = len(grid_list[x_before][y_before]) # before 칸에 있는 개체수
#         print(x_movement, y_movement)
        if type(grid_list[x_after][y_after]) == list: # 이동하려는 위치가 경계가 아니라면 :
            if x_movement != 0 or y_movement != 0 : # 이동을 한다면:
                if len(grid_list[x_before][y_before]) > 1: # 기존에 있던 위치에 다른 사람이 있다면 :
                    # 칸에 있는 사람 중 자신보다 늦게 온 사람들에 대해서:
                    for j in range(index_before + 1, how_many_in_before):
                        # index를 1씩 줄인다
                        id_for_reduce = grid_list[x_before][y_before][j][0] # index를 줄여야 하는 개체의 id
                        lst_temp = list(id_coordindex[id_for_reduce])
                        lst_temp[1] -= 1
                        id_coordindex[id_for_reduce] = tuple(lst_temp)
                # 목적지에 개체 append
                grid_list[x_after][y_after].append(id_status)
                # 시작지에서 개체 remove
                grid_list[x_before][y_before].remove(id_status)
#                 print(x_after, y_after)
#                 print(id_coordindex)
#                 print(index_after)
                index_after = len(grid_list[x_after][y_after]) - 1 # 이동한 위치에서의 index
                id_coordindex[i] = ((x_after, y_after), index_after) # 쌍방향 dict update
#                 print(id_coordindex[i])
#                 print(id_coordindex)
#         print()

In [45]:
# move_uniform()
# print(grid_list)
# print(id_coordindex)
# print(len(id_coordindex))
# print_NbyM(grid_list)

In [46]:
def print_NbyM(grid_list):
    for i in range(len(grid_list)):
        for j in range(len(grid_list[0])):
            if type(grid_list[i][j]) != list:
                print("000", end="     ")
            elif len(grid_list[i][j]) == 0:
                print("---", end="     ")
            else:
                if len(grid_list[i][j]) > 1:
                    for k in range(len(grid_list[i][j])):
                        print("%d,%d" % (grid_list[i][j][k][0],grid_list[i][j][k][1]), end=" ")
                else:
                    print("%d,%d" % (grid_list[i][j][0][0],grid_list[i][j][0][1]), end="     ")
        print()

In [47]:
def findSusceptible(id_num):
    global grid_list
    susceptible = []
    close_contact = []
    # 입력된 식별번호를 가지는 (감염된) 개체의 좌표 저장
    x = id_coordindex[id_num][0][0]
    y = id_coordindex[id_num][0][1]
    # 상하좌우 대각선으로 한 칸 이내의 공간에 대해서:
    for a in range(-1, 2):
        for b in range(-1, 2):
            if a == 0 and b == 0: # 제자리를 확인한다면:
                if len(grid_list[x][y]) > 1: # 자신 말고 다른 개체가 있다면:
                    for i in set(range(len(grid_list[x][y]))) - {id_coordindex[id_num][1]}:
                        if grid_list[x + a][y + b][i][1] == 0: # 해당 개체가 Susceptible 상태라면:
                            id_close_contact = grid_list[x][y][i][0] # 밀접접촉한 개체의 id
                            close_contact.append(id_close_contact) # close_contact(list)에 id를 추가
            elif type(grid_list[x + a][y + b]) == list: # 그 외 경우 중 경계가 아니라면:
                for i in range(len(grid_list[x + a][y + b])): # 해당 칸에 존재하는 개체 수만큼:
                    id_susceptible = grid_list[x + a][y + b][i][0]
                    if grid_list[x + a][y + b][i][1] == 0: # 해당 개체가 Susceptible 상태라면:
                        susceptible.append(id_susceptible) # susceptible(list)에 id를 추가
    return susceptible, close_contact

In [48]:
def infect(alpha, alpha_close_contact):
    global grid_list
    global active
    global total_infected
    for i in range(id_coordindex.__len__()):
        x, y = id_coordindex[i][0][0], id_coordindex[i][0][1] # 처음의 x, y좌표
        index = id_coordindex[i][1] # 처음의 위치에서의 index
        status = grid_list[x][y][index][1] # 개체의 상태(status)
        if status == 1: # i번 개체가 감염되었다면:
            susceptible, close_contact = findSusceptible(i)
            for id_person in susceptible:
                # alpha의 확률로 상태를 infected(1)로 변경
                if random.uniform(0, 1) <= alpha:
                    x_temp, y_temp = id_coordindex[id_person][0][0], id_coordindex[id_person][0][1]
                    index_temp = id_coordindex[id_person][1]
                    grid_list[x_temp][y_temp][index_temp][1] = 1
                    active += 1
                    total_infected += 1
            for id_person in close_contact:
                # alpha_close_contact의 확률로 상태를 infected(1)로 변경
                if random.uniform(0, 1) <= alpha_close_contact:
                    x_temp, y_temp = id_coordindex[id_person][0][0], id_coordindex[id_person][0][1]
                    index_temp = id_coordindex[id_person][1]
                    grid_list[x_temp][y_temp][index_temp][1] = 1 # 상태를 infected로 변경
                    active += 1
                    total_infected += 1

In [49]:
def recover(beta):
    global grid_list
    global active
    global recovered
    for i in range(id_coordindex.__len__()):
        x, y = id_coordindex[i][0][0], id_coordindex[i][0][1] # 처음의 x, y좌표
        index = id_coordindex[i][1] # 처음의 위치에서의 index
        status = grid_list[x][y][index][1] # 개체의 상태(status)
        if status == 1: # i번 개체가 감염되었다면:
            # beta의 확률로 상태를 recovered(2)로 변경
            if random.uniform(0, 1) <= beta:
                x_temp, y_temp = id_coordindex[i][0][0], id_coordindex[i][0][1]
                index_temp = id_coordindex[i][1]
                grid_list[x_temp][y_temp][index_temp][1] = 2
                recovered += 1
                active -= 1

In [50]:
def elapse_ndays_movement(N, M, days, alpha, alpha_close_contact, beta):
    global grid_list
    grid_list = scatter_uniform(N, M)
    mid_x, mid_y = (N + 1) // 2, (M + 1) // 2
    grid_list[mid_x][mid_y][0][1] = 1 # 최초 감염자
    for i in range(days + 1):
        print("Day", i)
        print_NbyM(grid_list)
        print("Total Infected: ", total_infected)
        print("Active: ", active)
        print("Recovered: ", recovered)
        print()
        move_uniform()
        recover(beta)
        infect(alpha, alpha_close_contact)

In [52]:
elapse_ndays_movement(5, 5, 5, 0.5, 0.6, 0.2)

Day 0
000     000     000     000     000     000     000     
000     3,0     7,0     ---     ---     ---     000     
000     ---     ---     ---     1,0     ---     000     
000     ---     ---     0,1     ---     8,0     000     
000     ---     2,0     ---     5,0     ---     000     
000     ---     6,0     4,0     ---     ---     000     
000     000     000     000     000     000     000     
Total Infected:  1
Active:  1
Recovered:  0

Day 1
000     000     000     000     000     000     000     
000     3,0     7,0     ---     ---     ---     000     
000     ---     ---     ---     1,1     8,0     000     
000     ---     ---     2,1 5,0 0,1     ---     000     
000     ---     ---     ---     ---     ---     000     
000     ---     6,0     4,0     ---     ---     000     
000     000     000     000     000     000     000     
Total Infected:  3
Active:  3
Recovered:  0

Day 2
000     000     000     000     000     000     000     
000     3,0     ---     7,0     ---  